<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=2cbe8e7772581705578e73f72a5ac3612ab6a1b9c3aab147840df386d8eb9c99
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66


   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-22 13:45:06
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  3.11 L
Current:  1.43 C
-------------------
Today PnL: -68.43 K (-0.48%)
Current PnL: -17.44 L (-11.73%)
CY Booked + Current PnL: -6.00 L (-4.04%)
-------------------
Total profit:  1.91 L
Total loss:  -19.35 L
-------------------
Total Booked + Current PnL: 20.98 L (17.24%)
Total Booked PnL: 38.42 L (31.57%)
Curr Year Booked PnL: 11.44 L (8.02%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.26 C
Est FTT PnL: 82.95 L (58.15%)
Deployed:  1.22 C
Current:  1.43 C
CAGR/XIRR %: 9.78%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BLUESTARCO,2080.00,15.76,69.0,H-MC,3.52,199810.0,35140.0,8192.0,1.75,21.34,4.10,26.31,89.0,4.29,1.43,31.03,X40N,NTT,AC
76,TTKPRESTIG,770.00,99.57,45.0,M-SC,3.33,85969.0,-14808.0,14898.0,-1.21,-14.69,17.33,0.09,245.0,-0.99,0.62,12.94,OX40N,NTT,DURABLES
49,NATIONALUM,244.55,-43.66,66.0,H-MC,8.58,112307.0,11803.0,17307.0,0.14,11.74,15.41,28.96,79.0,0.68,0.80,49.70,MH,ATH,METALS
56,RELIANCE,1533.00,-12.54,54.0,H-LC,4.82,218899.0,8533.0,20248.0,-0.30,4.06,9.25,13.68,37.0,0.42,1.57,21.25,XY25,NTT,REFINERIES
41,ITC,452.00,-38.34,46.0,H-LC,2.28,199219.0,-919.0,21814.0,-0.79,-0.46,10.95,10.44,4.0,-0.04,1.43,5.08,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SHALBY,327.00,1277.99,81.0,M-SC,6.51,187463.0,6320.0,39142.0,12.45,3.49,20.88,25.10,235.0,0.16,1.34,50.23,XY24,NTT,HEALTHCARE
8,AWL,485.00,-63.48,64.0,H-MC,3.92,260238.0,-41513.0,202934.0,6.05,-13.76,77.98,53.50,116.0,-0.20,1.86,15.59,XY24,NTT,FMCG
80,VAIBHAVGBL,521.00,64.19,66.0,H-SC,6.51,142371.0,-40404.0,151469.0,2.06,-22.11,106.39,60.77,125.0,-0.27,1.02,27.52,XR,NTT,JEWELLERY
31,HINDZINC,730.22,23.40,63.0,M-LC,1.68,200840.0,-4236.0,116809.0,1.91,-2.07,58.16,54.89,52.0,-0.04,1.44,21.53,X5K,ATH,METALS
34,ICICIPRULI,790.00,-19.50,53.0,H-MC,2.17,138561.0,2773.0,39975.0,1.75,2.04,28.85,31.48,107.0,0.07,0.99,14.41,X40,ATH,INSURANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,LTIM,7230.20,-4.05,51.0,H-LC,1.44,237712.0,-8439.0,87644.0,-4.12,-3.43,36.87,32.18,16.0,-0.10,1.70,31.38,X200,ATH,IT
53,RAJESHEXPO,518.00,1793.94,53.0,L-SC,2.44,51680.0,-85497.0,85587.0,-3.71,-62.33,165.61,0.07,267.0,-1.00,0.37,28.53,OX40N,NTT,JEWELLERY
83,VOLTAS,1530.00,-1.64,47.0,H-MC,2.74,206250.0,14508.0,23244.0,-3.16,7.57,11.27,19.69,99.0,0.62,1.48,14.88,XY25,NTT,AC
81,VALIANTORG,838.00,-319.59,48.0,H-SC,6.01,122094.0,-41511.0,162825.0,-3.10,-25.37,133.36,74.15,139.0,-0.25,0.87,56.50,XR,NTT,CHEMICALS
73,TCS,4476.75,-27.40,44.0,H-LC,12.25,288909.0,-57047.0,131916.0,-3.02,-16.49,45.66,21.64,1.0,-0.43,2.07,2.35,X40,ATH,IT


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,IEX,219.00,-29.69,54.0,H-SC,16.79,200904.0,950.0,96273.0,-0.66,0.48,47.92,48.63,115.0,0.01,1.44,12.08,XR,NTT,MISC
29,HAVELLS,2069.16,0.48,59.0,H-MC,1.94,249148.0,1417.0,73648.0,0.40,0.57,29.56,30.30,92.0,0.02,1.79,13.94,X40,ATH,ELECTRICAL


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,UJJIVANSFB,60.0,93.70,51.0,H-SC,9.31,122175.0,-20304.0,39829.0,-1.18,-14.25,32.60,13.70,163.0,-0.51,0.88,44.20,OX40N,NTT,BANKS
76,TTKPRESTIG,770.0,99.57,45.0,M-SC,3.33,85969.0,-14808.0,14898.0,-1.21,-14.69,17.33,0.09,245.0,-0.99,0.62,12.94,OX40N,NTT,DURABLES
44,KANSAINER,340.0,-67.06,62.0,H-SC,0.82,226809.0,-42858.0,79202.0,0.06,-15.89,34.92,13.47,138.0,-0.54,1.63,14.72,XY24,NTT,PAINTS
18,CERA,9475.0,-20.29,46.0,H-SC,2.14,145475.0,-30428.0,72447.0,0.97,-17.30,49.80,23.89,149.0,-0.42,1.04,25.64,OX40N,NTT,CERAMICS
71,TATAELXSI,9161.0,-21.02,47.0,H-MC,7.27,106799.0,-25036.0,67262.0,-1.77,-18.99,62.98,32.03,98.0,-0.37,0.77,19.62,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BLUESTARCO,2080.0,15.76,69.0,H-MC,3.52,199810.0,35140.0,8192.0,1.75,21.34,4.1,26.31,89.0,4.29,1.43,31.03,X40N,NTT,AC


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SHALBY,327.00,1277.99,81.0,M-SC,6.51,187463.0,6320.0,39142.0,12.45,3.49,20.88,25.10,235.0,0.16,1.34,50.23,XY24,NTT,HEALTHCARE
59,SAIL,228.00,49.16,71.0,M-MC,8.21,235139.0,10177.0,157473.0,0.26,4.52,66.97,74.53,192.0,0.06,1.69,38.24,XY24,BTT,STEEL
14,BLUESTARCO,2080.00,15.76,69.0,H-MC,3.52,199810.0,35140.0,8192.0,1.75,21.34,4.10,26.31,89.0,4.29,1.43,31.03,X40N,NTT,AC
29,HAVELLS,2069.16,0.48,59.0,H-MC,1.94,249148.0,1417.0,73648.0,0.40,0.57,29.56,30.30,92.0,0.02,1.79,13.94,X40,ATH,ELECTRICAL
17,CAMS,5211.76,-1.17,59.0,H-SC,3.10,112420.0,10416.0,33512.0,-2.09,10.21,29.81,43.06,122.0,0.31,0.81,29.08,X40N,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,NATIONALUM,244.55,-43.66,66.0,H-MC,8.58,112307.0,11803.0,17307.0,0.14,11.74,15.41,28.96,79.0,0.68,0.80,49.70,MH,ATH,METALS
84,WHIRLPOOL,2270.00,-41.17,48.0,M-SC,0.82,99435.0,7936.0,70818.0,-0.90,8.67,71.22,86.07,223.0,0.11,0.71,44.40,XR,NTT,DURABLES
11,BANKINDIA,190.00,-27.95,67.0,H-MC,6.61,185944.0,6136.0,106267.0,0.36,3.41,57.15,62.52,88.0,0.06,1.33,38.66,XR,NTT,BANKS
36,INDIAMART,4850.92,-53.52,32.0,H-SC,11.35,129834.0,6498.0,127263.0,-0.81,5.27,98.02,108.45,119.0,0.05,0.93,28.80,AR,ATH,MISC
85,WIPRO,420.00,-11.52,49.0,M-LC,5.68,155384.0,4439.0,105009.0,-2.06,2.94,67.58,72.51,53.0,0.04,1.11,9.14,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VBL,671.64,-17.33,39.0,H-LC,6.85,297882.0,-17960.0,130621.0,-1.59,-5.69,43.85,35.67,5.0,-0.14,2.13,7.38,X40N,ATH,FMCG
30,HCLTECH,1943.91,-1.08,42.0,H-LC,9.83,226080.0,-15844.0,79105.0,-1.87,-6.55,34.99,26.15,8.0,-0.20,1.62,6.84,X40,ATH,IT
73,TCS,4476.75,-27.40,44.0,H-LC,12.25,288909.0,-57047.0,131916.0,-3.02,-16.49,45.66,21.64,1.0,-0.43,2.07,2.35,X40,ATH,IT
19,CIPLA,1795.00,-20.81,46.0,H-LC,5.49,211610.0,7110.0,34302.0,-1.98,3.48,16.21,20.25,10.0,0.21,1.52,12.51,X40N,BTT,PHARMA
41,ITC,452.00,-38.34,46.0,H-LC,2.28,199219.0,-919.0,21814.0,-0.79,-0.46,10.95,10.44,4.0,-0.04,1.43,5.08,X40,NTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,LTIM,7230.2,-4.05,51.0,H-LC,1.44,237712.0,-8439.0,87644.0,-4.12,-3.43,36.87,32.18,16.0,-0.10,1.70,31.38,X200,ATH,IT
41,ITC,452.0,-38.34,46.0,H-LC,2.28,199219.0,-919.0,21814.0,-0.79,-0.46,10.95,10.44,4.0,-0.04,1.43,5.08,X40,NTT,FMCG
64,SIEMENS,4671.5,2.10,59.0,H-LC,3.57,164010.0,-22085.0,69573.0,-1.37,-11.87,42.42,25.51,15.0,-0.32,1.18,20.86,AR,ATH,ELECTRICAL
50,NESTLEIND,1377.0,-9.37,51.0,H-LC,4.26,278811.0,13385.0,43411.0,-0.25,5.04,15.57,21.40,11.0,0.31,2.00,12.74,XY25,NTT,FMCG
56,RELIANCE,1533.0,-12.54,54.0,H-LC,4.82,218899.0,8533.0,20248.0,-0.30,4.06,9.25,13.68,37.0,0.42,1.57,21.25,XY25,NTT,REFINERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4476.75,-27.40,44.0,H-LC,12.25,288909.0,-57047.0,131916.0,-3.02,-16.49,45.66,21.64,1.0,-0.43,2.07,2.35,X40,ATH,IT
41,ITC,452.00,-38.34,46.0,H-LC,2.28,199219.0,-919.0,21814.0,-0.79,-0.46,10.95,10.44,4.0,-0.04,1.43,5.08,X40,NTT,FMCG
79,UNITDSPR,1644.00,-11.80,57.0,H-LC,7.65,231168.0,-2976.0,51597.0,1.19,-1.27,22.32,20.77,86.0,-0.06,1.66,5.56,X40N,NTT,BREWERIES
30,HCLTECH,1943.91,-1.08,42.0,H-LC,9.83,226080.0,-15844.0,79105.0,-1.87,-6.55,34.99,26.15,8.0,-0.20,1.62,6.84,X40,ATH,IT
82,VBL,671.64,-17.33,39.0,H-LC,6.85,297882.0,-17960.0,130621.0,-1.59,-5.69,43.85,35.67,5.0,-0.14,2.13,7.38,X40N,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SIEMENS,4671.50,2.10,59.0,H-LC,3.57,164010.0,-22085.0,69573.0,-1.37,-11.87,42.42,25.51,15.0,-0.32,1.18,20.86,AR,ATH,ELECTRICAL
41,ITC,452.00,-38.34,46.0,H-LC,2.28,199219.0,-919.0,21814.0,-0.79,-0.46,10.95,10.44,4.0,-0.04,1.43,5.08,X40,NTT,FMCG
19,CIPLA,1795.00,-20.81,46.0,H-LC,5.49,211610.0,7110.0,34302.0,-1.98,3.48,16.21,20.25,10.0,0.21,1.52,12.51,X40N,BTT,PHARMA
5,ASIANPAINT,3465.66,-13.06,48.0,H-LC,7.75,219569.0,-32199.0,85412.0,0.54,-12.79,38.90,21.13,27.0,-0.38,1.57,18.11,X40,ATH,PAINTS
56,RELIANCE,1533.00,-12.54,54.0,H-LC,4.82,218899.0,8533.0,20248.0,-0.30,4.06,9.25,13.68,37.0,0.42,1.57,21.25,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4476.75,-27.40,44.0,H-LC,12.25,288909.0,-57047.0,131916.0,-3.02,-16.49,45.66,21.64,1.0,-0.43,2.07,2.35,X40,ATH,IT
39,INFY,2275.00,-18.16,48.0,H-LC,8.20,318478.0,5590.0,166086.0,-2.92,1.79,52.15,54.87,3.0,0.03,2.28,8.55,X40,BTT,IT
41,ITC,452.00,-38.34,46.0,H-LC,2.28,199219.0,-919.0,21814.0,-0.79,-0.46,10.95,10.44,4.0,-0.04,1.43,5.08,X40,NTT,FMCG
82,VBL,671.64,-17.33,39.0,H-LC,6.85,297882.0,-17960.0,130621.0,-1.59,-5.69,43.85,35.67,5.0,-0.14,2.13,7.38,X40N,ATH,FMCG
1,ABB,7934.00,-37.53,68.0,H-LC,9.39,260971.0,-648.0,119864.0,-0.00,-0.25,45.93,45.57,7.0,-0.01,1.87,11.05,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,7083.33,67.0,L-SC,5.17,78845.0,-14965.0,91586.0,0.68,-15.95,116.16,81.67,269.0,-0.16,0.57,52.69,XR,NTT,CERAMICS
21,COFFEEDAY,80.00,-43.66,49.0,L-SC,42.35,83482.0,-30067.0,70117.0,-1.89,-26.48,83.99,35.27,268.0,-0.43,0.60,104.32,XR,NTT,HOTELS
60,SAMMAANCAP,326.00,-173.86,67.0,M-SC,3.92,85098.0,-17112.0,110500.0,-0.96,-16.74,129.85,91.37,208.0,-0.15,0.61,38.14,XY25,NTT,FINANCE
3,ALKYLAMINE,4546.37,-5.68,44.0,H-SC,12.30,87849.0,-13114.0,107641.0,-1.68,-12.99,122.53,93.63,148.0,-0.12,0.63,30.85,SR,ATH,CHEMICALS
48,MASFIN,398.61,-16.89,52.0,H-SC,12.62,94500.0,-3480.0,25080.0,-0.11,-3.55,26.54,22.05,152.0,-0.14,0.68,37.36,XR,ATH,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SHALBY,327.00,1277.99,81.0,M-SC,6.51,187463.0,6320.0,39142.0,12.45,3.49,20.88,25.10,235.0,0.16,1.34,50.23,XY24,NTT,HEALTHCARE
70,TANLA,1963.11,-15.40,72.0,H-SC,13.44,226486.0,-44275.0,374223.0,-1.80,-16.35,165.23,121.86,133.0,-0.12,1.62,77.18,AR,ATH,IT
60,SAMMAANCAP,326.00,-173.86,67.0,M-SC,3.92,85098.0,-17112.0,110500.0,-0.96,-16.74,129.85,91.37,208.0,-0.15,0.61,38.14,XY25,NTT,FINANCE
80,VAIBHAVGBL,521.00,64.19,66.0,H-SC,6.51,142371.0,-40404.0,151469.0,2.06,-22.11,106.39,60.77,125.0,-0.27,1.02,27.52,XR,NTT,JEWELLERY
7,ATULAUTO,844.00,3759.46,57.0,M-SC,4.95,120242.0,-27126.0,78939.0,-0.89,-18.41,65.65,35.16,236.0,-0.34,0.86,22.82,XY24,NTT,AUTO


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.61
1,25,44.18
2,50,75.32


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.85
LC    31.12
MC    24.00
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.98
X40      14.94
X40N     11.62
XR       11.44
AR        8.56
XY25      8.40
OX40N     7.98
X200      1.70
MH        1.67
X5K       1.44
SR        1.24
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.90
H-LC    24.55
H-MC    21.01
M-SC    13.41
M-LC     5.52
M-MC     2.69
L-SC     1.54
L-LC     1.05
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
IT,13.11,-14.79,77.38
FMCG,12.40,-3.71,40.82
FINANCE,10.21,-15.16,65.40
MISC,7.21,-14.40,78.01
BANKS,6.19,-15.43,76.15
PAINTS,5.73,-15.40,32.60
ELECTRICAL,5.64,-4.80,41.57
INSURANCE,3.75,-5.51,44.95
AC,3.62,4.11,11.94


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3056506.0,22
XR,1311525.0,14
AR,1136428.0,9
X40,762871.0,10
X40N,583823.0,9
OX40N,535588.0,10
XY25,376518.0,6
SR,247906.0,2
X5K,116809.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3620235.0,29
M-SC,1372869.0,17
H-MC,1172044.0,15
H-LC,1101833.0,14
M-LC,379657.0,4
M-MC,303363.0,2
L-SC,247290.0,3
L-MC,60401.0,1
L-LC,37729.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1212762.0      6
M-SC       XY24       783371.0      7
H-SC       AR         782996.0      5
           XR         763166.0      7
H-MC       XY24       538931.0      4
H-LC       X40        484333.0      5
M-MC       XY24       303363.0      2
H-SC       X40N       295972.0      4
           OX40N      254937.0      4
           SR         247906.0      2
H-LC       X40N       216520.0      3
H-MC       X40        206316.0      4
H-LC       AR         189437.0      2
H-MC       XY25       164630.0      2
M-SC       AR         163995.0      2
L-SC       XR         161703.0      2
M-LC       XY24       157839.0      2
M-SC       OX40N      127802.0      4
M-LC       X5K        116809.0      1
M-SC       XR         114979.0      2
           XY25       110500.0      1
H-MC       XR         106267.0      1
M-LC       XR         105009.0      1
H-LC       X200        87644.0      1
L-SC       OX40N       85587.0      1
M-SC       X40         72222.0      1
H-MC       X40N        71331.0      2
           OX40N       67262.0      1
H-LC       XY25        63659.0      2
H-SC       MH          62496.0      1
L-MC       XR          60401.0      1
H-LC       XY24        60240.0      1
L-LC       XY25        37729.0      1
H-MC       MH          17307.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 29.0 seconds
